In [1]:
!git clone https://github.com/lseventeen/RF-UNet.git

Cloning into 'RF-UNet'...
remote: Enumerating objects: 147, done.
remote: Counting objects: 100% (144/144), done.
remote: Compressing objects: 100% (88/88), done.
remote: Total 147 (delta 60), reused 125 (delta 49), pack-reused 3
Receiving objects: 100% (147/147), 281.78 MiB | 23.17 MiB/s, done.
Resolving deltas: 100% (60/60), done.
Updating files: 100% (22/22), done.


In [8]:
!pip install albumentations==0.5.2
!pip install bunch==1.0.1
!pip install loguru==0.5.3
!pip install matplotlib==3.3.2
!pip install numpy==1.20.3
!pip3 install opencv_python
!pip install Pillow==9.0.1
!pip install ruamel.base==1.0.0
!pip install scikit_learn==1.0.2
!pip install segmentation==0.2.2
!pip install timm==0.3.2
!pip install torch==1.7.1+cu110 torchvision==0.8.2+cu110 torchaudio==0.7.2 -f https://download.pytorch.org/whl/torch_stable.html
!pip install torchstat==0.0.7
!pip install torchvision==0.8.1
!pip install tqdm==4.61.1
!pip install ttach==0.0.3
!pip install yacs==0.1.8
!pip install ruamel.yaml

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
   ---------------------------------------- 0.0/14.9 MB ? eta -:--:--
   ---------------------------------------- 0.0/14.9 MB 1.3 MB/s eta 0:00:12
   -- ------------------------------------- 1.0/14.9 MB 12.3 MB/s eta 0:00:02
   -------------- ------------------------- 5.3/14.9 MB 42.5 MB/s eta 0:00:01
   -------------------------------- ------- 12.1/14.9 MB 131.2 MB/s eta 0:00:01
   ---------------------------------------- 14.9/14.9 MB 131.1 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.20.3
    Uninstalling numpy-1.20.3:
      Successfully uninstalled numpy-1.20.3


  You can safely remove it manually.


Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
     ---------------------------------------- 0.0/13.7 MB ? eta -:--:--
     --------------------------------------- 0.0/13.7 MB 435.7 kB/s eta 0:00:32
      --------------------------------------- 0.3/13.7 MB 3.5 MB/s eta 0:00:04
     -------- ------------------------------- 3.0/13.7 MB 23.8 MB/s eta 0:00:01
     ------------------------ --------------- 8.5/13.7 MB 45.2 MB/s eta 0:00:01
     ----------------------------------- -- 12.7/13.7 MB 108.8 MB/s eta 0:00:01
     -------------------------------------- 13.7/13.7 MB 109.9 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.24.4
    Uninstalling numpy-1.24.4:
      Successfully uninstalled numpy-1.24.4


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
scikit-image 0.21.0 requires numpy>=1.21.1, but you have numpy 1.20.3 which is incompatible.


Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


...

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
     ---------------------------------------- 0.0/133.0 kB ? eta -:--:--
     --------- ----------------------------- 30.7/133.0 kB 1.3 MB/s eta 0:00:01
     -------------------------------------- 133.0/133.0 kB 1.9 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'error'
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in links: https://download.pytorch.org/whl/torch_stable.html
     ---------------------------------------- 0.0/2.1 GB ? eta -:--:--
     ---------------------------------------- 0.0/2.1 GB 28.1 MB/s eta 0:01:13
     ---------------------------------------- 0.0/2.1 GB 35.6 MB/s eta 0:00:58
     ---------------------------------------- 0.0/2.1 GB 34.5 MB/s eta 0:01:00
     ---------------------------------------- 0.0/2.1 GB 41.1 MB/s e

  You can safely remove it manually.
  You can safely remove it manually.


Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


ERROR: Ignored the following yanked versions: 0.1.6, 0.1.7, 0.1.8, 0.1.9, 0.2.0, 0.2.1, 0.2.2, 0.2.2.post2, 0.2.2.post3, 0.11.0, 0.15.0
ERROR: Could not find a version that satisfies the requirement torchvision==0.8.1 (from versions: 0.5.0, 0.9.0, 0.9.1, 0.10.0, 0.10.1, 0.11.1, 0.11.2, 0.11.3, 0.12.0, 0.13.0, 0.13.1, 0.14.0, 0.14.1, 0.15.1, 0.15.2, 0.16.0, 0.16.1, 0.16.2)
ERROR: No matching distribution found for torchvision==0.8.1


Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


In [6]:
!pip install tensorboard==2.6.0
!pip install protobuf==3.20.0

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
     ---------------------------------------- 0.0/904.4 kB ? eta -:--:--
     - ----------------------------------- 30.7/904.4 kB 660.6 kB/s eta 0:00:02
     ------------------------------------  890.9/904.4 kB 11.3 MB/s eta 0:00:01
     -------------------------------------- 904.4/904.4 kB 9.5 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 4.25.1
    Uninstalling protobuf-4.25.1:
      Successfully uninstalled protobuf-4.25.1


  You can safely remove it manually.


In [5]:
class DictToObject:
    def __init__(self, dictionary):
        for key, value in dictionary.items():
            setattr(self, key, value)

In [1]:
import argparse
from bunch import Bunch
from loguru import logger
from ruamel.yaml import YAML

from torch.utils.data import DataLoader
import sys
sys.path.append('repo_paper\RF-UNet')
import models
from dataset import vessel_dataset
from trainer import Trainer
from utils import losses
from utils.helpers import get_instance, seed_torch

In [2]:
# !nvidia-smi
import torch
print(torch.rand(2,3).cuda())

tensor([[0.8969, 0.5913, 0.7354],
        [0.0053, 0.0736, 0.1613]], device='cuda:0')


In [3]:
def main(CFG, data_path, batch_size, with_val=False):
    seed_torch()
    if with_val:
        train_dataset = vessel_dataset(data_path, mode="training", split=0.9)
        val_dataset = vessel_dataset(
            data_path, mode="training", split=0.9, is_val=True)
        val_loader = DataLoader(
            val_dataset, batch_size, shuffle=False, num_workers=16, pin_memory=True, drop_last=False)
    else:
        train_dataset = vessel_dataset(data_path, mode="training")
        print(train_dataset)
    train_loader = DataLoader(
        train_dataset, batch_size, shuffle=True, num_workers=16, pin_memory=True, drop_last=True)

    logger.info('The patch number of train is %d' % len(train_dataset))
    model = get_instance(models, 'model', CFG)
    logger.info(f'\n{model}\n')
    loss = get_instance(losses, 'loss', CFG)
    trainer = Trainer(
        model=model,
        loss=loss,
        CFG=CFG,
        train_loader=train_loader,
        val_loader=val_loader if with_val else None
    )

    trainer.train()


if __name__ == '__main__':
    # parser = argparse.ArgumentParser()
    # parser.add_argument('-dp', '--dataset_path', default="Project_data\\RETINA\\training_pro", type=str,
    #                     help='the path of dataset')
    # parser.add_argument('-bs', '--batch_size', default=16,
    #                     help='batch_size for trianing and validation')
    # parser.add_argument("--val", help="split training data for validation",
    #                     required=False, default=False, action="store_true")
    # args = parser.parse_args()

    with open('repo_paper\\RF-UNet\\config.yaml', encoding='utf-8') as file:
        yaml = YAML(typ='safe')
        CFG = yaml.load(file)
        # CFG = DictToObject(CFG)
    main(CFG, "A:\\EIM\\Project_data\\RETINA", 128,True )


2023-12-19 20:02:00.741 | INFO     | __main__:main:15 - The patch number of train is 21870
2023-12-19 20:02:00.806 | INFO     | __main__:main:17 - 
FR_UNet(
  (block1_3): block(
    (fuse): feature_fuse(
      (conv11): Conv2d(1, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (conv33): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (conv33_di): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(2, 2), dilation=(2, 2), bias=False)
      (norm): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (conv): conv(
      (conv): Sequential(
        (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): Dropout2d(p=0.2, inplace=False)
        (3): LeakyReLU(negative_slope=0.1, inplace=True)
        (4): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias

RuntimeError: CUDA out of memory. Tried to allocate 512.00 MiB (GPU 0; 4.00 GiB total capacity; 19.51 GiB already allocated; 0 bytes free; 19.59 GiB reserved in total by PyTorch)

In [2]:
from PIL import Image
import pickle
import matplotlib.pyplot as plt
with open('Project_data\\RETINA\\training_pro\\img_patch_1.pkl', 'rb') as file:
    lista_patch = pickle.load(file)
    list_of_arrays = lista_patch.squeeze().tolist()


In [21]:
import yaml

# Path to your YAML file
yaml_file_path = 'repo_paper\RF-UNet\config.yaml'

# Open the YAML file and load its contents
with open(yaml_file_path, 'r') as file:
    yaml_data = yaml.safe_load(file)

# Display or work with the loaded YAML data
print(yaml_data['amp'])


True


In [ ]:
from PIL import Image
import pickle
import matplotlib.pyplot as plt
with open('Project_data\\RETINA\\training_pro\\img_patch_1.pkl', 'rb') as file:
    lista_patch = pickle.load(file)
